## Setup and utilities


In [ ]:

import os
import tarfile
import urllib.request as request

import numpy as np

np.set_printoptions(precision=4, suppress=True)

def degree_matrix(A):
    d = A.sum(axis=1)
    return np.diag(d)
import numpy as np

def laplacian(A, normalized=False):
    D = np.diag(A.sum(axis=1))

    if normalized:
        D_inv_sqrt = np.diag(1.0 / np.sqrt(np.diag(D) + 1e-10))
        I = np.eye(A.shape[0])
        return I - np.dot(np.dot(D_inv_sqrt, A), D_inv_sqrt)
    else:
        return D - A


def scaled_laplacian(L):

    lambda_max = np.linalg.eigvalsh(L).max()
    I = np.eye(L.shape[0])
    return (2 / lambda_max) * L - I

def get_largest_eigenvalue(mat):
    return float(np.linalg.eigvalsh(mat).max())


## 1. Load the Cora dataset
The Cora dataset represents academic publications and citations, consisting of 2708 nodes (papers) and 5429 edges (citations). Each node has a binary word vector of size 1433 where each value represents the absence/presence of a specific word. The publications are assigned to one of seven classes, representing the field of the publication. Here use the dataset to predict the field of papers.

![Visualization of the Cora dataset](https://graphsandnetworks.com/wp-content/uploads/2019/09/CoraBalloons.png)

In [ ]:
URL = "https://linqs-data.soe.ucsc.edu/public/lbc/cora.tgz"
DATA_DIR = "./"
TGZ_PATH = os.path.join(DATA_DIR, "cora.tgz")

os.makedirs(DATA_DIR, exist_ok=True)

### Download
if not os.path.exists(TGZ_PATH):
    print("Downloading Cora dataset...")
    request.urlretrieve(URL, TGZ_PATH)
else:
    print("File already exists:", TGZ_PATH)

### Extract
with tarfile.open(TGZ_PATH, "r:gz") as tar:
    tar.extractall(path=DATA_DIR)
print("Extracted to", DATA_DIR)

### Load cora.content, which contains nodes, features and labels
content_path = os.path.join(DATA_DIR, "cora", "cora.content")
ids = []
features = []
labels_raw = []
with open(content_path, "r") as f:
    for line in f:
        parts = line.strip().split()
        if len(parts) < 3:
            continue
        ids.append(parts[0])
        features.append([float(x) for x in parts[1:-1]])
        labels_raw.append(parts[-1])

ids = np.array(ids)
X = np.array(features, dtype=float) # matrix of dimensions (samples,features)

### Label encoding
classes = sorted(set(labels_raw))
label_map = {c: i for i, c in enumerate(classes)}
y = np.array([label_map[c] for c in labels_raw], dtype=int)

### Load cora.cites, which contains the edges, and store it in an adjacency matrix
n = len(ids)
# For each id (node identifiers given by the dataset), get the index (idx)
id_to_idx = {pid: i for i, pid in enumerate(ids)}
A = np.zeros((n, n), dtype=float)

cites_path = os.path.join(DATA_DIR, "cora", "cora.cites")
with open(cites_path, "r") as f:
    for line in f:
        parts = line.strip().split()
        if len(parts) != 2:
            continue
        src, dst = parts
        if src in id_to_idx and dst in id_to_idx:
            i, j = id_to_idx[src], id_to_idx[dst]
            A[i, j] = 1.0
            A[j, i] = 1.0  # undirected

print("Nodes:", n, "Features:", X.shape[1], "Classes:", len(classes))
print("Adjacency shape:", A.shape)


/tmp/ipython-input-3533348425.py:16: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
  tar.extractall(path=DATA_DIR)


Extracted to ./
Nodes: 2708 Features: 1433 Classes: 7
Adjacency shape: (2708, 2708)


## 1. Laplacian and scaled Laplacian
The Laplacian $L=D-A$ contains both the adjacency structure as well as the degree of nodes, essentially describing the whole graph. The normalization $L = I - D^{-1/2} A D^{-1/2}$ has a number of favorable properties mostly relevant for spectral graph theory; for the case of graph neural networks, the normalization can mostly be seen as reducing the impact of high-degree nodes relative to other nodes.


In [ ]:
L = laplacian(A, normalized=True)
L_tilde = scaled_laplacian(L)


## 2. Chebyshev polynomials on graphs
Chebyshev polynomials are used for approximations where uniform accuracy over the whole domain [-1,1] is relevant (compared to Taylor series, which are locally accurate).


In [ ]:

def chebyshev_polynomials(L_tilde, X, K):
    Ts = [X]  # T0 = X

    if K >= 1:
        T1 = np.dot(L_tilde, X)
        Ts.append(T1)

        for _ in range(2, K + 1):
            T2 = 2 * np.dot(L_tilde, Ts[-1]) - Ts[-2]
            Ts.append(T2)

    return Ts


for K in [0,1,2,3]:
    Ts = chebyshev_polynomials(L_tilde, X, K)
    print(f"K={K}", [t.shape for t in Ts]) # Should be the same shape for every K, but +1 entry.


K=0 [(2708, 1433)]
K=1 [(2708, 1433), (2708, 1433)]
K=2 [(2708, 1433), (2708, 1433), (2708, 1433)]
K=3 [(2708, 1433), (2708, 1433), (2708, 1433), (2708, 1433)]


## 3. ChebNet layer
The idea of ChebNet is that since convolution in the graph domain corresponds to the element-wise product in the spectral domain (Eigenvalue decomposition; $h*g = U\hat GU^Th$), the spectral filters $\hat g_v$ for nodes $v$ that make up $\hat G$ can be approximated. These approximations are parameterized with learned coefficients $\theta$.


In [ ]:
def chebnet_forward_pass(TX, thetas):

    Z = np.zeros((TX[0].shape[0], thetas[0].shape[1]))

    for T_k, theta_k in zip(TX, thetas):
        Z += np.dot(T_k, theta_k)

    return Z

### For demo: Map the input feature dimension to the output feature dimension of 10
F_in, F_out = X.shape[1], 10
K = 2
thetas_init = [np.random.randn(F_in, F_out)*0.1 for _ in range(K+1)]

### Compute polynomials once, then perform a forward pass
TX = chebyshev_polynomials(L_tilde, X, K)
Z_demo = chebnet_forward_pass(TX, thetas_init)
print("Z_demo shape:", Z_demo.shape) #


Z_demo shape: (2708, 10)


## 4. Semi-supervised split
Use a small labeled set per class for training, hold-out validation, and the rest for test. As the nodes that are neither in training, validation, nor test set still influence the training through the parameter K (order of polynomials & receptive field), this split of the dataset can be seen as semi-supervised.

In [ ]:
def canonical_cora_split(y, train_per_class, val_size, test_size, seed=42):
    rng = np.random.RandomState(seed)
    n = len(y)
    classes = np.unique(y)

    # 20 labeled nodes per class
    train_idx = []
    for c in classes:
        idx_c = np.where(y == c)[0]
        rng.shuffle(idx_c)
        take = min(train_per_class, len(idx_c))
        train_idx.append(idx_c[:take])
    train_idx = np.concatenate(train_idx)

    # From the remaining, sample 500 val and 1000 test
    remaining = np.setdiff1d(np.arange(n), train_idx, assume_unique=False)
    rng.shuffle(remaining)

    val_take = min(val_size, len(remaining))
    val_idx = remaining[:val_take]

    remaining2 = remaining[val_take:]
    test_take = min(test_size, len(remaining2))
    test_idx = remaining2[:test_take]

    return train_idx, val_idx, test_idx

### Cora has n=2708 and 7 classes
train_idx, val_idx, test_idx = canonical_cora_split(y, 20, 500, 1000)
print(len(train_idx), len(val_idx), len(test_idx))

140 500 1000


## 6. Train single-layer ChebNet with manual gradients
For the forward pass, compute $Z=\sum_k T_k(\tilde L)X\theta_k$. Then compute the gradients on $Z$

Forward: Z = sum_k T_k X Theta_k; Softmax; Cross-entropy on labeled nodes only. Backprop only to Theta_k. Precompute T_k X.


In [ ]:

def one_hot(y, C):
    Y = np.zeros((len(y), C), dtype=float)
    Y[np.arange(len(y)), y] = 1.0
    return Y

def softmax(logits):
    logits = logits - logits.max(axis=1, keepdims=True)
    e = np.exp(logits)
    return e / (e.sum(axis=1, keepdims=True) + 1e-12)

def accuracy(logits, y_true):
    return float((logits.argmax(axis=1) == y_true).mean())

def compute_grads(TX, Thetas, idx, y_true):
    logits = chebnet_forward_pass(TX, Thetas)
    P = softmax(logits)
    Y = one_hot(y_true, P.shape[1])
    G = np.zeros_like(P)
    G[idx] = (P[idx] - Y)
    grads = [TX[k].T @ G for k in range(len(Thetas))]
    loss = -np.mean(np.log(P[idx, y_true] + 1e-12))
    return grads, loss, logits

def step_adam(params, grads, m, v, t, lr=0.01, beta1=0.9, beta2=0.999, eps=1e-8, weight_decay=5e-4):
    t += 1
    for i in range(len(params)):
        g = grads[i] + weight_decay*params[i]
        m[i] = beta1*m[i] + (1-beta1)*g
        v[i] = beta2*v[i] + (1-beta2)*(g*g)
        mhat = m[i] / (1-beta1**t)
        vhat = v[i] / (1-beta2**t)
        params[i] -= lr * mhat / (np.sqrt(vhat) + eps)
    return params, m, v, t

### Hyperparameters
K = 2
epochs = 200
patience = 20

TX = chebyshev_polynomials(L_tilde, X, K)

Fin = X.shape[1]; Fout = int(y.max()+1)
thetas = [np.random.randn(Fin, Fout)*0.01 for _ in range(K+1)]
momentum = [np.zeros_like(t) for t in thetas]
velocity = [np.zeros_like(t) for t in thetas]

### Initialization
t_adam = 0
best_val = -1e9
best_params = None
patience_left = patience

### Training loop
for epoch in range(1, epochs+1):

    grads, train_loss, train_logits = compute_grads(TX, thetas, train_idx, y[train_idx])
    thetas, momentum, velocity, t_adam = step_adam(thetas, grads, momentum, velocity, t_adam)

    train_acc = accuracy(train_logits[train_idx], y[train_idx])

    val_logits = chebnet_forward_pass(TX, thetas)
    P_val = softmax(val_logits)
    val_loss = -np.mean(np.log(P_val[val_idx, y[val_idx]] + 1e-12))
    val_acc = accuracy(val_logits[val_idx], y[val_idx])


    if epoch % 10 == 0 or epoch == 1:
        print(f"Epoch {epoch:03d} | train {train_loss:.4f}/{train_acc:.3f} | val {val_loss:.4f}/{val_acc:.3f}")

    if val_acc > best_val:
        best_val, best_params, patience_left = val_acc, [p.copy() for p in thetas], patience
    else:
        patience_left -= 1
        if patience_left == 0:
            break

if best_params is not None:
    thetas = best_params

test_logits = chebnet_forward_pass(TX, thetas)
test_acc = accuracy(test_logits[test_idx], y[test_idx])
print("Test accuracy:", round(test_acc, 4))


Epoch 001 | train 1.9536/0.114 | val 1.8454/0.468
Epoch 010 | train 0.0899/1.000 | val 1.2710/0.640
Epoch 020 | train 0.0088/1.000 | val 1.1449/0.634
Epoch 030 | train 0.0031/1.000 | val 1.1238/0.636
Test accuracy: 0.626


## 7. The influence of K
In addition to reducing the approximation error by allowing bigger polynomials, the parameter $K$ determines from how many hops (jumps from one node to another) away information is aggregated (receptive field). It is analogous to the number of message passing steps in spatial GNNs such as GraphSAGE.




In [ ]:

def train_chebnet_K(K, lr=0.02, epochs=150):
    Lt = scaled_laplacian(laplacian(A))
    TX = chebyshev_polynomials(Lt, X, K)
    Fin = X.shape[1]
    Fout = int(y.max()+1)
    Thetas = [np.random.randn(Fin, Fout) * 0.01 for _ in range(K+1)]
    momentum = [np.zeros_like(t) for t in Thetas]
    velocity = [np.zeros_like(t) for t in Thetas]
    t_adam = 0; best_val = -1e9; best = None; patience_left = 15
    for epoch in range(1, epochs+1):
        grads, train_loss, train_logits = compute_grads(TX, Thetas, train_idx, y[train_idx])
        Thetas, momentum, velocity, t_adam = step_adam(Thetas, grads, momentum, velocity, t_adam, lr=lr)

        train_acc = accuracy(train_logits[train_idx], y[train_idx])

        val_logits = chebnet_forward_pass(TX, Thetas)
        P_val = softmax(val_logits)
        val_loss = -np.mean(np.log(P_val[val_idx, y[val_idx]] + 1e-12))
        val_acc = accuracy(val_logits[val_idx], y[val_idx])


        if val_acc > best_val:
            best_val, best, patience_left = val_acc, [t.copy() for t in Thetas], 15
        else:
            patience_left -= 1
            if patience_left == 0:
                break
    if best is not None:
        Thetas = best
    test_acc = accuracy(chebnet_forward_pass(TX, Thetas)[test_idx], y[test_idx])
    return best_val, test_acc

for K in range(20):
    val_acc, test_acc = train_chebnet_K(K)
    print(f"K={K} -> val={val_acc:.3f}, test={test_acc:.3f}")


K=0 -> val=0.476, test=0.480
K=1 -> val=0.490, test=0.498
K=2 -> val=0.530, test=0.527
K=3 -> val=0.570, test=0.572
K=4 -> val=0.610, test=0.600
K=5 -> val=0.644, test=0.643
K=6 -> val=0.692, test=0.679
K=7 -> val=0.702, test=0.700
K=8 -> val=0.706, test=0.709
K=9 -> val=0.722, test=0.734
K=10 -> val=0.716, test=0.735
K=11 -> val=0.706, test=0.721
K=12 -> val=0.708, test=0.719
K=13 -> val=0.694, test=0.704
K=14 -> val=0.724, test=0.715
K=15 -> val=0.714, test=0.708
K=16 -> val=0.718, test=0.713
K=17 -> val=0.710, test=0.705
K=18 -> val=0.702, test=0.694
K=19 -> val=0.702, test=0.690
